In [1]:
hail = []
with open("input.txt", "rt") as f:
    for line in f.read().strip().split("\n"):
        p, v = line.split(" @ ")
        p = list(map(int, p.split(", ")))
        v = list(map(int, v.split(", ")))
        hail.append((p, v))

X = 0
Y = 1
Z = 2

# Part 1

In [2]:
from itertools import combinations
from typing import Optional, TypeAlias

In [3]:
Point: TypeAlias = tuple[int, int, int]
Line: TypeAlias = tuple[int, int, int]


def get_line_coefficients(p: Point, q: Point) -> tuple[int, int, int]:
    a = p[Y] - q[Y]
    b = q[X] - p[X]
    c = q[X] * p[Y] - p[X] * q[Y]
    return a, b, c


hail_lines = []
for crystal in hail:
    p, v = crystal
    q = p[X] + v[X], p[Y] + v[Y], p[Z] + v[Z]
    hail_lines.append(get_line_coefficients(p, q))


def get_line_intersection(a: Line, b: Line) -> Optional[Point]:
    d = a[0] * b[1] - a[1] * b[0]

    if d != 0:
        dx = a[2] * b[1] - a[1] * b[2]
        dy = a[0] * b[2] - a[2] * b[0]

        x = dx / d
        y = dy / d
        return x, y
    return None


X_MIN = 200000000000000
X_MAX = 400000000000000
Y_MIN = X_MIN
Y_MAX = X_MAX

s = 0
for (hail_a, line_a), (hail_b, line_b) in combinations(list(zip(hail, hail_lines)), 2):
    intersection = get_line_intersection(line_a, line_b)

    if intersection is not None:
        correct_range = (
            intersection[X] > X_MIN
            and intersection[X] <= X_MAX
            and intersection[Y] > Y_MIN
            and intersection[Y] <= Y_MAX
        )

        pa, va = hail_a
        pb, vb = hail_b
        in_the_past = (
            (va[X] > 0 and pa[X] >= intersection[X])
            or (va[X] < 0 and pa[X] <= intersection[X])
            or (va[Y] > 0 and pa[Y] >= intersection[Y])
            or (va[Y] < 0 and pa[Y] <= intersection[Y])
            or (vb[X] > 0 and pb[X] >= intersection[X])
            or (vb[X] < 0 and pb[X] <= intersection[X])
            or (vb[Y] > 0 and pb[Y] >= intersection[Y])
            or (vb[Y] < 0 and pb[Y] <= intersection[Y])
        )

        if correct_range and not in_the_past:
            s += 1
s

28266

# Part 2

`> pip install z3-solver`

In [4]:
import z3

In [5]:
s = z3.Ints("sx sy sz")
vs = z3.Ints("vsx vsy vsz")

solver = z3.Solver()
for i, (p, vp) in enumerate(hail):
    ti = z3.Int(f"t{i}")
    solver.add(ti > 0)
    solver.add(s[X] + vs[X] * ti == p[X] + vp[X] * ti)
    solver.add(s[Y] + vs[Y] * ti == p[Y] + vp[Y] * ti)
    solver.add(s[Z] + vs[Z] * ti == p[Z] + vp[Z] * ti)
solver.check()

solver.model().evaluate(s[X] + s[Y] + s[Z]).as_long()

786617045860267